# Energy Reconstruction Using CNN

In [3]:
import numpy as np
import tensorflow as tf
import keras
import os

from csv import writer
from sklearn import preprocessing
from keras import layers, models, callbacks
from functional_data_tools import load_preprocessed, dataPrep

## FIX THIS!
simPrefix = '/Users/kmays/simFiles'

Load & view simulation data

In [4]:
#load x (icetop tank grid) & y(the event's info)
#x is a 2D array/list & y is a dictionary with list values
x, y = load_preprocessed(simPrefix, 'train')
#play with these as needed to better understand the data
print(x.shape)
print(y.keys())

KeyboardInterrupt: 

In [3]:
#temp = x[x==0]
#x[x==0]=1E-5
#print(x[x==0])
assert not np.any(np.isnan(x))


Split y into its values & prep data

In [4]:
#y is a dictionary of energy, composition, & direction info
energy = y['energy']
comp = y['comp']
theta, phi = y['dir'].transpose()
nevents = len(energy)
# 85/15 split for training/validation
trainCut = (np.random.uniform(size=nevents) < 0.85)
testCut = np.logical_not(trainCut)

Check & customize as needed for each model

In [9]:
# Name for model to prevent overwriting
#specify file location
key = 'baseline'
i=0
while(os.path.isfile('trainedModels/{}.h5'.format(key+str(i)))):
    i = i + 1
key=key+str(i)
print(key)

# specify data prep values & number of epochs (early stopping will kick in)
prep = {'q':None, 't':False, 'normed':True, 'reco':'small', 'cosz':True}
numepochs=100

# Establish arrays to be trained on
x_i = dataPrep(x, y, **prep)
temp_y = energy

#temp = x[...,...]
#preprocessing.normalize(temp)
print(np.average(x_i[0]))
print(np.average(temp_y))
#x_i[np.isnan(x_i[0])]
#return the first two or last two dim then go from there

baseline0
0.019400259034327343
7.016269463840849


Customize sequential model layers

In [10]:
# Create sequential model
charge_input = keras.Input(shape=(10,10,2,))

conv1_layer = layers.Conv2D(64,kernel_size=3,padding='same',activation='relu')(charge_input)

conv2_layer = layers.Conv2D(32,kernel_size=3,padding='same',activation='relu')(conv1_layer)

conv3_layer = layers.Conv2D(16, kernel_size=3, padding='same',activation="relu")(conv2_layer)

flat_layer = layers.Flatten()(conv3_layer)
zenith_input=keras.Input(shape=(1,))
concat_layer = layers.Concatenate()([flat_layer,zenith_input])

dense1_layer = layers.Dense(256,activation='relu')(concat_layer)

dense2_layer = layers.Dense(256,activation='relu')(dense1_layer)

dense3_layer = layers.Dense(256,activation="relu")(dense2_layer)

output = layers.Dense(1)(dense3_layer)

model = models.Model(inputs=[charge_input,zenith_input],outputs=output,name=key)

# Compile model & specify 
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae','mse'])

In [11]:
model.summary()

Model: "baseline0"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 10, 10, 2)]  0                                            
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 10, 10, 64)   1216        input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 10, 10, 32)   18464       conv2d_3[0][0]                   
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 10, 10, 16)   4624        conv2d_4[0][0]                   
__________________________________________________________________________________________

Train & customize callbacks here

In [14]:
#record performance at each epoch(csv), include earlystopping, then train model
csv_logger = callbacks.CSVLogger('trainedModels/{}.csv'.format(key))
early_stop = callbacks.EarlyStopping(patience=5, restore_best_weights=True) # default -> val_loss
#checkpoint = callbacks.ModelCheckpoint('trainedModels/checkpoints/%s.h5' % key,save_best_only=True)
callbacklist = [early_stop, csv_logger]
history = model.fit({"input_3":x_i[0],"input_4":x_i[1].reshape(-1,1)}, temp_y, epochs=numepochs,validation_split=0.15,callbacks=callbacklist)

Epoch 1/100
 1040/14354 [=>............................] - ETA: 9:19 - loss: 0.8866 - mae: 0.6546 - mse: 0.8866

KeyboardInterrupt: 

Saving mechanism (comment out csv part if needed)

In [ ]:
# Save model(h5)
model.save('model_%s.h5' % key)

#retrieve best metrics for this model & create a new row/list
num_epoch=len(history.history['loss'])
val_loss=np.min(history.history['val_loss'])
index=history.history['val_loss'].index(val_loss)
loss=history.history(['loss'][index])
mae=history.history(['mae'][index])
mse=history.history(['loss'][index])
val_mae=history.history(['val_mae'][index])
val_mse=history.history(['val_mse'][index])
new_row=[key,num_epoch, loss, mae, mse, val_loss, val_mae, val_mse]

#save info after every trained model to be compared to others (results.csv)
with open('trainedModels/results.csv', 'a') as f_object:
    csv_writer=writer(f_object)
    csv_writer.writerow(new_row)
f_object.close()